<a href="https://colab.research.google.com/github/KBVBrahmarao/datasciencecoursera/blob/main/Lab4_InLab2_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB4:
In-Lab:
2. Develop a MapReduce program for sorting key–value pairs in ascending order.
a) Extract keys from input (e.g., names, numbers, IDs).
b) Implement a Mapper that emits (key, value).
c) Use a custom comparator (if needed).
d) Run and verify that output is fully sorted.

In [1]:
#STEP 1 — Install Java
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-17-jdk-headless is already the newest version (17.0.17+10-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [2]:
#Check Java:
!java -version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [3]:
#STEP 2 — Set Java Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-17-openjdk-amd64/bin"

In [4]:
#STEP 3 — Download and Extract Hadoop
!wget -q https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /content/hadoop

In [5]:
#STEP 4 — Set Hadoop Environment Variables
os.environ["HADOOP_HOME"] = "/content/hadoop"
os.environ["HADOOP_CONF_DIR"] = "/content/hadoop/etc/hadoop"
os.environ["PATH"] += ":/content/hadoop/bin:/content/hadoop/sbin"

In [6]:
#Extra STEP 1 — Set JAVA_HOME again
!export HADOOP_HOME=/content/Hadoop
!export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
!export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

In [7]:
#Extra Step 2 — Verify directories
!ls /content/Hadoop/bin

ls: cannot access '/content/Hadoop/bin': No such file or directory


In [8]:
#ExtraSTEP 3 — Check Hadoop version again
!export HADOOP_HOME=/content/Hadoop && export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64 && export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin && hadoop version

ERROR: Cannot execute /content/Hadoop/libexec/hadoop-config.sh.


In [9]:
#STEP 5 — Correct Hadoop Configuration Files - core-site.xml
%%writefile /content/hadoop/etc/hadoop/core-site.xml
<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>file:///</value>
    </property>
</configuration>

Overwriting /content/hadoop/etc/hadoop/core-site.xml


In [10]:
#Step 6 - Create hdfs-site.xml (even though HDFS cannot run in Colab)
#Step 6 - Create hdfs-site.xml (even though HDFS cannot run in Colab)
%%writefile /content/hadoop/etc/hadoop/hdfs-site.xml
<?xml version="1.0"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>

Overwriting /content/hadoop/etc/hadoop/hdfs-site.xml


In [11]:
#STEP  7A — Verify that files are correct
!cat /content/hadoop/etc/hadoop/core-site.xml
!cat /content/hadoop/etc/hadoop/hdfs-site.xml

<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>file:///</value>
    </property>
</configuration>
<?xml version="1.0"?>
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>


In [12]:
#Step 7B - Verify the files were created
!ls /content/hadoop/etc/hadoop

capacity-scheduler.xml		  kms-log4j.properties
configuration.xsl		  kms-site.xml
container-executor.cfg		  log4j.properties
core-site.xml			  mapred-env.cmd
hadoop-env.cmd			  mapred-env.sh
hadoop-env.sh			  mapred-queues.xml.template
hadoop-metrics2.properties	  mapred-site.xml
hadoop-policy.xml		  shellprofile.d
hadoop-user-functions.sh.example  ssl-client.xml.example
hdfs-rbf-site.xml		  ssl-server.xml.example
hdfs-site.xml			  user_ec_policies.xml.template
httpfs-env.sh			  workers
httpfs-log4j.properties		  yarn-env.cmd
httpfs-site.xml			  yarn-env.sh
kms-acls.xml			  yarnservice-log4j.properties
kms-env.sh			  yarn-site.xml


In [13]:
#STEP 7C — Set correct config directory
%%bash
export HADOOP_HOME=/content/hadoop
export HADOOP_CONF_DIR=/content/hadoop/etc/hadoop
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export PATH=$PATH:$JAVA_HOME/bin:$HADOOP_HOME/bin:$HADOOP_HOME/sbin

hadoop version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /content/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


In [ ]:
#STEP 7D — Delete old output
!rm -rf filteroutput

In [14]:
#NEXT IMPORTANT STEPs
#STEP 7 - Fix log4j warning (optional but recommended)
%%bash
cat > /content/hadoop/etc/hadoop/log4j.properties <<'EOF'
log4j.rootLogger=INFO, console
log4j.appender.console=org.apache.log4j.ConsoleAppender
log4j.appender.console.target=System.err
log4j.appender.console.layout=org.apache.log4j.PatternLayout
log4j.appender.console.layout.ConversionPattern=%d{ISO8601} %-5p %c: %m%n
EOF

In [15]:
#STEP 8 — Upload input file
from google.colab import files
uploaded = files.upload()

Saving Lab4_InLab2_Data.txt to Lab4_InLab2_Data.txt


In [28]:
# Mapper Code in Python
def mapper(line):
    line = line.strip()
    if not line:
        return None

    fields = line.split()     # split by space
    if len(fields) < 2:
        return None

    name = fields[0]
    salary = int(fields[1])
    return (name, salary)

In [29]:
#Run Mapping
with open("Lab4_InLab2_Data.txt", "r") as f:
    mapped_output = [mapper(line) for line in f]

# Remove None values
mapped_output = [x for x in mapped_output if x is not None]

print("Mapped Output:", mapped_output)

Mapped Output: [('Amit', 87), ('John', 56), ('David', 92), ('Riya', 75), ('Vani', 60)]


In [30]:
#Shuffle & Sort by key
sorted_output = sorted(mapped_output, key=lambda x: x[0])
print("Sorted Output:", sorted_output)

Sorted Output: [('Amit', 87), ('David', 92), ('John', 56), ('Riya', 75), ('Vani', 60)]


In [31]:
#Reduce (identity reducer)
def reducer(sorted_pairs):
    for key, value in sorted_pairs:
        print(f"{key}\t{value}")

reducer(sorted_output)

Amit	87
David	92
John	56
Riya	75
Vani	60
